In [1]:
# !pip install -qU accelerate
# !pip install -qU diffusers
!pip install -qU gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.3 MB/s eta 0:00:00


In [2]:
import gradio as gr
import numpy as np
from PIL import Image
import torch

from diffusers import AutoPipelineForInpainting

import warnings
warnings.filterwarnings('ignore')

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

cuda


In [3]:
gr.__version__

'5.6.0'

In [4]:
inpaint_pipe_sd15_ip = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    torch_dtype=torch.float16,
    # 사전에 이미 fp16으로 변환되어있는 가중치를 로드하는 옵션
    # 주지 않으면 실시간으로 fp16으로 변환
    variant="fp16"
).to(device)


model_index.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

text_encoder/config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

scheduler/scheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

...

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
def inpaint(img, ctx):
    # img: btn.click에서 지정한 input 컴포넌트가 매칭
    # img["layers"]: 사용자가 마우스르 그린 그림
    # img["background"]: 사용자가 업로드한 바탕 그림
    # img["composite"]: 바탕 그림과 사용자가 마우스로 그린 그림이 함께 있는 그림

    #############################################################################
    # 마스크 이미지 처리
    # 0번 레이어에서 마지막 채널을 가져옴, (H, W, C, A)
    alpha_channel = img["layers"][0][:, :, 3]
    # 0이면 원래 검은색, 사용자가 그린 부분은 255
    mask = np.where(alpha_channel == 0, 0, 255)
    # 3채널로 확장 (H, W, 1) -> (H, W, 3)
    mask_img = np.stack([mask] * 3, axis=-1).astype(np.uint8)

    ##############################################################################
    # 배경 이미지 처리
    # 배경 이미지에서 알파 채널 제거 (H, W, 4) -> (H, W, 3)
    base_img = img["background"][:, :, :3]

    # 이미지로 저장하고 하면 쿠다 메모리 에러 안나고 잘됨
    # 여기서 numpy->PIL로 바로 바꿔서 하면 잘될 것 같음
    base_img = Image.fromarray(base_img)
    mask_img = Image.fromarray(mask_img)

    if ctx == '':
        prompt = "seamless background continuation, PLEASE draw NO NEW OBJECTS and TEXTS !!!"
    else:
        prompt = ctx #"A small robot, high resolution, sitting on a park bench"

    # generator = torch.Generator(device).manual_seed(0)
    inpaint_image = inpaint_pipe_sd15_ip(
        prompt=prompt,
        image=base_img,
        mask_image=mask_img,
        # generator=generator
    ).images[0]

    # 이 함수의 리턴이 btn.click의 output component에 연결
    return base_img, mask_img, inpaint_image


In [6]:
with gr.Blocks() as demo:
    with gr.Column():
        with gr.Row():
            input_img = gr.ImageMask(
                sources=["upload"], layers=False, transforms=[],
                format="png", label="base image", show_label=True)
            inpaint_img = gr.Image(label='inpainted image')
        with gr.Row():
            original_img = gr.Image(label='original image')
            mask_img = gr.Image(label="mask image", show_label=True)

    ctx = gr.Textbox(label="Prompt")

    btn = gr.Button()
    btn.click(
        inpaint, # 클릭했을 때 실행될 함수
        [input_img, ctx],   # input component
        [original_img, mask_img, inpaint_img] # output component
    )

demo.launch(height=1000,debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7021012fd88db8ce40.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7021012fd88db8ce40.gradio.live
